<a href="https://colab.research.google.com/github/Bhuvan588/Agribot/blob/main/Agribot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
ds = pd.read_csv("questionsv4.csv", engine='python')

In [ ]:
ds.head()

,questions,answers
0,asking about the control measure for aphid inf...,suggested him to spray rogor@2ml/lit.at evenin...
1,asking about the control measure of flower dro...,suggested him to apply fertilizer in recommend...
2,asking about how to avail kisan credit card lo...,suggested to consult with officer-marketing an...
3,asking about source of early ahu rice variety,suggested to take early ahu rice variety from ...
4,asking that he has not got proper friut from h...,suggested him to aplly recommended fertilizer ...


In [ ]:
ds.isnull().sum()

questions     0
answers      53
dtype: int64

In [ ]:
ds= ds.dropna(subset=['answers'])

In [ ]:
ds.isnull().sum()

questions    0
answers      0
dtype: int64

In [ ]:
ds.shape

(67873, 2)

In [ ]:
print(len(ds[ds["answers"] == "explained in details"]))

1217


NLP Preprocessing Steps

In [ ]:
import inflect
p = inflect.engine()
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


# Download NLTK resources (may need internet connection first time)
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


def convert_number(text):
  # Split string into a list (temporary for digit manipulation)
  temp_str = text.split()
  new_string = []  # Empty list to store modified words

  for word in temp_str:
    if word.isdigit():
      temp = p.number_to_words(word)
      new_string.append(temp)  # Append converted number as word
    else:
      new_string.append(word)  # Append word as it is

  # Join the words back into a string
  text = ' '.join(new_string)
  return text


def remove_punctuation(text):
  translator = str.maketrans('', '', string.punctuation)
  return text.translate(translator)


# Remove whitespace from text
def remove_whitespace(text):
  return " ".join(text.split())  # Join words after splitting by whitespace


def remove_stopwords(text):
  stop_words = set(stopwords.words("english"))
  word_tokens = word_tokenize(text)
  filtered_text = [word for word in word_tokens if word not in stop_words]
  # Join the filtered words back into a string
  return ' '.join(filtered_text)


# Define a function to lemmatize text
def lemmatize_text(text):
  # Tokenize the text (split into words)
  tokens = nltk.word_tokenize(text)  # No need to call lower() here
  tokens = [word.lower() for word in tokens]  # Convert each word to lowercase

  # Create a WordNet lemmatizer
  lemmatizer = WordNetLemmatizer()

  # Lemmatize each word, considering its part-of-speech (optional)
  lemmas = [lemmatizer.lemmatize(word) for word in tokens]

  # Return the lemmatized text
  return " ".join(lemmas)


def preprocess(text):
  res = str(text)  # Ensure res is a string


  res = res.lower()  # Convert to lowercase


  res = remove_punctuation(res)  # Remove punctuation


  res = remove_whitespace(res)  # Remove whitespace


  res = convert_number(res)  # Convert numbers to words






  res = remove_stopwords(res)  # Remove stopwords


  res = lemmatize_text(res)  # Lemmatize text


  return res

input_str = "suggested to apply bordeaux mixture. apply borax @ 50 gm/plant per year along with urea & ssp."

preprocess(input_str)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'suggested apply bordeaux mixture apply borax fifty gmplant per year along urea ssp'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

user_query = "Fertlizers required for corn?"
bot_response1 = "Corn typically thrives with nitrogen-rich fertilizers. Have you considered using Ammonium Sulfate?"
bot_response2 = "Weeds can significantly reduce corn yield. Luckily, there are various herbicides available for pre- and post-emergence weed control."


# Preprocess sentences
user_query = preprocess(user_query)
bot_response1 = preprocess(bot_response1)
bot_response2 = preprocess(bot_response2)

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on all sentences
sentences = [user_query, bot_response1, bot_response2]
vectors = vectorizer.fit_transform(sentences)

# Calculate cosine similarity between user query and each bot response
query_vec = vectors[0]  # First element is the user query vector
similarity_scores = cosine_similarity(query_vec.reshape(1, -1), vectors[1:])  # Compare with bot responses

# Find the bot response with the highest similarity score
most_similar_idx = similarity_scores.argmax()

# Provide the corresponding bot response
chosen_response = sentences[most_similar_idx + 1]  # +1 to exclude user query

print(f"User Query: {user_query}")
print(f"Most Relevant Bot Response: {chosen_response}")

User Query: fertlizers required corn
Most Relevant Bot Response: corn typically thrives nitrogenrich fertilizer considered using ammonium sulfate


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



def find_most_similar_response(user_query):
  # Preprocess the user query
  user_query_processed = preprocess(user_query)

  # Load the dataset (ensure questions column is present)

  questions = ds["questions"]

  # Create TF-IDF vectorizer
  vectorizer = TfidfVectorizer()

  # Fit the vectorizer on all sentences (user query and questions)
  all_sentences = [user_query_processed] + questions.tolist()
  vectors = vectorizer.fit_transform(all_sentences)

  # Calculate cosine similarity between user query and each question
  query_vec = vectors[0]
  similarity_scores = cosine_similarity(query_vec.reshape(1, -1), vectors[1:])

  # Find the question with the highest similarity score
  most_similar_idx = similarity_scores.argmax()

  # Retrieve the corresponding answer (assuming an "answers" column exists)
  chosen_answer = ds["answers"].iloc[most_similar_idx] if "answers" in ds.columns else "Answer not found in dataset"

  if(len(chosen_answer.split())<=4):
    search_engine = "https://www.google.com/search?q=" + user_query_processed  # Replace with your preferred search engine
    print(f"Search the web for more details: {search_engine}")



  return chosen_answer

# Example usage
user_query = input("Enter your question: ")
answer = find_most_similar_response(user_query)
print(f"Agri Bot: {answer}")


Enter your question: regarding bordeaux
Agri Bot: spray bordeaux mixture @ 10 g per lit of water, 2 sprays at 15 days interval


In [ ]:
!pip install anvil-uplink

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
import anvil.server

anvil.server.connect("YOUR ANVIL UPLINK")

In [ ]:
@anvil.server.callable
def response_function(user_query):
    answer = find_most_similar_response(user_query)
    return answer

In [ ]:
anvil.server.wait_forever()

KeyboardInterrupt: 